# Импорт функции для загрузки данных из файла  

Функция принимает:  

+ путь до файла (строка)  
+ имя таблицы (строка)  
+ последний столбец для выгрузки (строка)  
+ столбец по которому объединяются таблицы, данные в столбце должны быть числовые
+ начальная/конечная строка с данными, по умолчанию определяется автоматически (кортеж)  


In [11]:
import string
import pandas as pd
import numpy as np
import os
def load_sheet(path_to_file: str, sheet_name: str, columns_range_header: str,\
        merge_by_column: str="A", rows_range: list=None)->pd.DataFrame:
    ''' Load sheet from excel file
    ----------
    Parameters:
    path_to_file : path to excel file (str)
    sheet_name : name of sheet from excel file (str)
    columns_range_header : header of the last column, max 702 columns (str)
    merge_by_column : column by which sheets will be merged (str), by default it is first column - "A", column should be numeric
    rows_range : list with start/end numeric data row indexes (int inside list), by default define range automatically
    -------
    Returns:
    pd_sheet : pandas DataFrame
    '''
    ### Проверяем что файл существует
    if not os.path.exists(path_to_file):
        error_message = f"Не нахожу такого файла: {path_to_file}"
        raise ValueError(error_message)
    ### Формируем заголовок для конечной таблицы
    header = os.path.basename(path_to_file)
    header = header.rsplit(".", maxsplit=1)[0]
    ### Create list with capital letters A,B,C ... J
    alphabetcnt = 26
    columns_name = list(string.ascii_uppercase[:alphabetcnt])
    for i in range(alphabetcnt):
        for j in range(alphabetcnt):
            columns_name.append(columns_name[i]+columns_name[j])
    ### Создаем числовый список нужных столбцов
    def columns_filter(col: list)->list:
        res = []
        for i in col:
            res.append(i)
            if i == columns_range_header:
                break
        return res
    columns_range = columns_filter(columns_name)
    ### Создаем числовый список нужных столбцов
    columns = [columns_name.index(x) for x in columns_range]
    ### Загружаем файл
    num_xlsx = pd.ExcelFile(path_to_file)
    pd_sheet = pd.read_excel(num_xlsx, sheet_name, header=None, usecols=columns)
    pd_sheet.columns = columns_range
    ### Делаем реиндекс как в Excel
    pd_sheet.set_index((i+1 for i in pd_sheet.index), inplace=True)
    ### Оставляем только нужные столбцы
    pd_sheet = pd_sheet.loc[:, columns_range]
    ### Конвертируем в числа и строки, не числа становятся NaN
    pd_sheet_num = pd.to_numeric(pd_sheet[merge_by_column], errors="coerce", downcast="integer")
    ### Определяем строки с данными в нужных столбцах по столбцу для объединений из numeric_columns
    rows_range = rows_range or [None, None]
    if rows_range[0] is None:
        rows_range[0] = pd_sheet_num.first_valid_index()
    if rows_range[0] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    if rows_range[1] is None:
        rows_range[1] = pd_sheet_num.last_valid_index()
    if rows_range[1] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    ### Удаляем лишние строки
    pd_sheet = pd_sheet.loc[rows_range[0]:rows_range[1],]
    ### Приводим типы к числовым еще раз, потому что иначе иногда остаются типы object
    # numeric_columns = columns_range
    # for i in numeric_columns:
        # pd_sheet[i] = pd.to_numeric(pd_sheet[i],downcast="integer")
    print(pd_sheet.head(2))
    print("....................")
    print(pd_sheet.tail(2).to_string(header=False))
    print(" ")
    return pd_sheet

# Загружаем настройки

In [9]:
### Настройки
imported_data = ("/home/hayer/git_projects/lfl_reports/data/LFL_август_2022.xlsx","TDSheet","E")
# rows_range = ((6, 6835),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442))

# Загружаем исходный файл


In [12]:
source_table = load_sheet(imported_data[0], imported_data[1], imported_data[2])

        A                                      B    C            D  \
4  107611  01. ЛКМ, клея, пропитки, растворители  NaN  95116.03833   
5  105124                      Клеевые материалы  NaN   4335.51216   

             E  
4  76952.34494  
5   3722.06992  
....................
5267  167674  Старый плинтус Korner и КорнерПласт  NaN  0  0
5268  167282                       Плинтус Korner  NaN  0  0
 


/home/hayer/git_projects/venv/pandas/lib64/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/hayer/git_projects/venv/pandas/lib64/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


# Шаблон сортировки

In [13]:
columns_sort_order = ["A", "E", "B", "D", "C" ]

# Сортировать столбцы в нужном порядке


In [14]:
result = source_table.loc[:, columns_sort_order]

# Скопировать результат в буфер обмена

In [15]:
result.to_clipboard(excel=True, index=False, float_format="%.2f", decimal=",")

A	E	B	D	C
107611	76952,34	01. ЛКМ, клея, пропитки, растворители	95116.03833	
105124	3722,07	Клеевые материалы	4335.51216	
330436	474,55	Cosmofen	461.03536	
310185	768,71	Ecolux	939.8958	
342121	90,01	Exclusive	212.48417	
109099	597,79	Henkel	721.8732	
344653	54,81	Poxipol	28.903180000000003	
199469	93,35	Quelyd	93.3077	
330435	3,77	TDM	39.59552	
346063	22,39	Алмаз	81.20721	
330437	145,17	Аметист	192.30727	
345442	97,79	Анлес	170.39396	
247700	519,20	Кволити	46.47274	
310184	742,51	Класс	1214.0492	
330438	0,00	Мастикс	0	
199576	17,94	Прочее	103.60285999999999	
320998	94,10	Секунда	30.383989999999997	
105775	1041,29	Колера Палиж	1421.29512	
307138	1014,85	Колера	1376.87686	
307137	26,44	Эмали	44.418260000000004	
106763	319,52	Краски Akzo Nobel	251.33933	
344543	1,97	Краска Dulux	0	
344542	62,88	Краска Hammerite	45.389100000000006	
356744	62,88	3в1	45.389100000000006	
344544	0,00	Краска Marshall	0	
307139	50,42	Лаки	19.284740000000003	
375886	0,00	Масло	0	
307140	204,26	Пропитки	186.66548